# Using KnowRob in Python

This notebook demonstrates how to use the KnowRob system directly in Python. It includes importing necessary modules, initializing the knowledge base, and executing queries.

### Importing KnowRob Modules

In [ ]:
import json
try:
	# This case works in ros1 environments
	from knowrob.kb import *
except ImportError:
	# If the import fails, import the knowrob.so directly
	from knowrob import *

First, we import the required modules from KnowRob. The `try-except` block ensures compatibility with different ROS environments, either using the ROS1-specific package or directly loading `knowrob.so`.

In [ ]:
InitKnowRob()

The `InitKnowRob()` function initializes the KnowRob system, setting up necessary configurations and connections.

### Setting Up Knowledge Base

In [ ]:
# Sample dictionary to be converted to JSON
sample_dict = {
	"logging": {
		"console-sink": {"level": "debug"},
		"file-sink": {"level": "debug"}
	},
	"semantic-web": {
		"prefixes": [
			{"alias": "swrl_test", "uri": "http://knowrob.org/kb/swrl_test"}
		]
	},
	"data-sources": [
		{"path": "owl/test/swrl.owl", "format": "rdf-xml"}
	],
	"data-backends": [
		{
			"type": "MongoDB",
			"name": "mongodb",
			"host": "localhost",
			"port": 27017,
			"db": "test",
			"read-only": False
		}
	],
	"reasoner": []
}
# Convert the dictionary to a JSON string
json_str = json.dumps(sample_dict)
# Initialize the KnowledgeBase with the PropertyTree
kb = KnowledgeBase(json_str)

This block defines the configuration for the KnowledgeBase, including logging, semantic web prefixes, data sources, and backends. The configuration is then serialized to a JSON string and used to initialize the `KnowledgeBase` instance.

### Submitting a Query

In [ ]:
phi1 = QueryParser.parse("swrl_test:hasAncestor(swrl_test:'Lea', ?y)")

Here, a query is parsed using the `QueryParser`. The query checks for ancestors of the entity `Lea` within the `swrl_test` namespace.

### Retrieving Query Results

In [ ]:
resultStream = kb.submitQueryFormula(phi1, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult1 = resultQueue.pop_front()

The query formulated in the previous step is submitted to the KnowledgeBase. The results are retrieved as a stream, and a queue is created to handle them.

### Processing Query Results


In [ ]:
if isinstance(nextResult1, AnswerYes):
    for substitution in nextResult1.substitution():
        variable = substitution[1]
        term = substitution[2]
        print(str(variable) + " : " + str(term))

This block checks if the result is affirmative (`AnswerYes`) and prints each substitution found in the query result, listing variable bindings.

### Negative Query Result Handling


In [ ]:
phi2 = QueryParser.parse("swrl_test:hasAncestor(swrl_test:'Lea', swrl_test:'Lea')")
resultStream = kb.submitQueryFormula(phi2, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult2 = resultQueue.pop_front()
if isinstance(nextResult2, AnswerNo):
    print("result is negative")

A second query checks for a specific condition, in this case, whether `Lea` is an ancestor of herself, which is expected to be false. The result is handled accordingly.

### Inconclusive Query Result Handling

In [ ]:
phi3 = QueryParser.parse("r(?x, ?y)")
resultStream = kb.submitQueryFormula(phi3, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult3 = resultQueue.pop_front()
if isinstance(nextResult3, AnswerDontKnow):
    print("We can't say if the result is true or false")


The final example demonstrates handling a situation where the system cannot determine the truth value of the query, resulting in an `AnswerDontKnow` response.